In [ ]:
import requests
from bs4 import *
arr_itog = []
url = 'https://music.yandex.ru/artist/3185166/info'
def parsing(url):
    global arr_itog
    html_cod = requests.get(url).text
    array = []
    bs = BeautifulSoup(html_cod)
    try:
      name_artist = bs.find('h1', attrs = {'class':'page-artist__title typo-h1 typo-h1_big'}).text
      array.append(name_artist)
    except:
      array.append('')
    regions_name_1= ['Москва и область','Санкт-Петербург и область','Краснодарский край','Республика Татарстан','Республика Башкортостан','Новосибирская область','Самарская область']
    try:
      M_total_count = bs.find_all('span', attrs = {'class':'artist-trends__total-count'})[0].text
      array.append(M_total_count)
    except:
      array.append('')
    try:
      M_total_count_likes = bs.find_all('span', attrs = {'class':'artist-trends__total-count'})[1].text
      array.append(M_total_count_likes)
    except:
      array.append('')
    try:
      recommend_all = bs.find('button',attrs={'class':'d-button deco-button deco-button-transparent d-button_rounded d-button_size_L d-button_w-icon d-button_w-icon-left'}).find('span',attrs = {'class': 'd-button__label'}).text
      array.append(recommend_all)
    except:
      array.append('')
    regions = {'Москва и область':0,'Санкт-Петербург и область':0,'Краснодарский край':0,'Республика Татарстан':0,'Республика Башкортостан':0,'Новосибирская область':0,'Самарская область':0}
    try:
      regions_html = bs.find('div',attrs = {'class':'page-artist__region-list page-artist__info-row_wide page-artist__section_regions'}).find_all('div', attrs = {'class':'page-artist__region'})
      for i in regions_html:
        region_name = i.find('span', attrs = {'class':'page-artist__region-caption typo'}).text
        count = i.find('span', attrs = {'class':'page-artist__region-count typo'}).text
        count = count.replace(' ', '')
        count = int(count)
        if region_name in regions.keys():
          regions[region_name] += count
      for i in regions_name_1:
        array.append(regions[i])
    except:
      pass
    links = []
    try:
      ss = bs.find('div',attrs = {'class':'page-artist__links-container page-artist__info-row_wide'}).find_all('span', attrs = {'class':'page-artist__link-caption'})
      links = [i.text for i in ss]
    except:
      pass
    Vk = 'ВКонтакте' in links
    Fb = 'Facebook' in links
    Tw = 'Twitter' in links
    Yt = 'YouTube' in links
    In = 'Instagram' in links
    array.append(Vk)
    array.append(Fb)
    array.append(Tw)
    array.append(Yt)
    array.append(In)
    arr_itog.append(array)
    return 0

array_links_str = []
url_1 = 'https://music.yandex.ru/genre/%D0%BC%D1%83%D0%B7%D1%8B%D0%BA%D0%B0%20%D0%B2%D1%81%D0%B5%D1%85%20%D0%B6%D0%B0%D0%BD%D1%80%D0%BE%D0%B2/artists?page='
for i in range(101):
  array_links_str.append(url_1+str(i))

for i in array_links_str:
    html_1 = requests.get(i).text
    bss = BeautifulSoup(html_1)
    url_base = 'https://music.yandex.ru'
    a_tegs = bss.find('div', attrs = {'class':'page-metatag__artists'}).find_all('a', attrs = {'class':'d-link deco-link'})
    links_artist = [url_base + i.attrs['href']+'/info' for i in a_tegs]
    for j in links_artist:
        parsing(j)
columns = ['name','M_total_count','M_total_count_likes','recommend_all','MSK','SPB','KRD','KZN','BASHK','NVSB','SAMARA','Vk','Fb','Tw','Yt','Inst']
import pandas as pd
import numpy as np
nda = np.array(arr_itog)
df = pd.DataFrame(nda)
df.columns = columns
df.to_csv('out.csv')